In [1]:
import os
import pandas as pd
import urllib
import ffmpeg
import moviepy.editor as mp
import cv2
import io
import PIL
import random
import math
import shutil
import pickle
import base64
from google.oauth2 import service_account
from google.cloud import vision
from PIL import Image

os.chdir("/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links")
LINK_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_links"
PARENT_DIR = "/nfs/sloanlab003/projects/arc_proj/story/data/sb_ad_vids"
df = pd.read_csv("nw_may_v1.csv")

from google.cloud import videointelligence_v1 as videointelligence
from google.cloud import vision

os.chdir("/nfs/sloanlab003/projects/arc_proj/story/code/nw/google-cloud-sdk")
GOOGLE_APPLICATION_CREDENTIALS = "/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json"
print('done')

done


## Google Video Intelligence (new)

### Label detection

In [2]:
def analyze_labels(path):
    from google.oauth2 import service_account
    from google.cloud import vision
    import io
    """Detect labels given a file path."""
    credentials = service_account.Credentials.from_service_account_file("/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json")
    video_client = videointelligence.VideoIntelligenceServiceClient(credentials=credentials)
    features = [videointelligence.Feature.LABEL_DETECTION]


    with io.open(path, "rb") as movie:
        input_content = movie.read()

    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content}
    )
    print("\nProcessing video for label annotations:")

    result = operation.result(timeout=90)
    print("\nFinished processing.")
    

    # Process video/segment level label annotations
    segment_labels = result.annotation_results[0].segment_label_annotations
    for i, segment_label in enumerate(segment_labels):
        print("Video label description: {}".format(segment_label.entity.description))
        for category_entity in segment_label.category_entities:
            print(
                "\tLabel category description: {}".format(category_entity.description)
            )

        for i, segment in enumerate(segment_label.segments):
            start_time = (
                segment.segment.start_time_offset.seconds
                + segment.segment.start_time_offset.microseconds / 1e6
            )
            end_time = (
                segment.segment.end_time_offset.seconds
                + segment.segment.end_time_offset.microseconds / 1e6
            )
            positions = "{}s to {}s".format(start_time, end_time)
            confidence = segment.confidence
            print("\tSegment {}: {}".format(i, positions))
            print("\tConfidence: {}".format(confidence))
        print("\n")

    # Process shot level label annotations
    shot_labels = result.annotation_results[0].shot_label_annotations
    for i, shot_label in enumerate(shot_labels):
        print("Shot label description: {}".format(shot_label.entity.description))
        print(type(shot_label.category_entities), len(shot_label.category_entities))
        for category_entity in shot_label.category_entities:
            print(
                "\tLabel category description: {}".format(category_entity.description)
            )

        for i, shot in enumerate(shot_label.segments):
            start_time = (
                shot.segment.start_time_offset.seconds
                + shot.segment.start_time_offset.microseconds / 1e6
            )
            end_time = (
                shot.segment.end_time_offset.seconds
                + shot.segment.end_time_offset.microseconds / 1e6
            )
            positions = "{}s to {}s".format(start_time, end_time)
            confidence = shot.confidence
            print("\tSegment {}: {}".format(i, positions))
            print("\tConfidence: {}".format(confidence))
        print("\n")

    # Process frame level label annotations
    frame_labels = result.annotation_results[0].frame_label_annotations
    for i, frame_label in enumerate(frame_labels):
        print("Frame label description: {}".format(frame_label.entity.description))
        for category_entity in frame_label.category_entities:
            print(
                "\tLabel category description: {}".format(category_entity.description)
            )

        # Each frame_label_annotation has many frames,
        # here we print information only about the first frame.
        frame = frame_label.frames[0]
        time_offset = frame.time_offset.seconds + frame.time_offset.microseconds / 1e6
        print("\tFirst frame time offset: {}s".format(time_offset))
        print("\tFirst frame confidence: {}".format(frame.confidence))
        print("\n")

### Logo Detection

In [3]:
def detect_logo(local_file_path):
    """Performs asynchronous video annotation for logo recognition on a local file."""
    
    credentials = service_account.Credentials.from_service_account_file("/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json")
    client = videointelligence.VideoIntelligenceServiceClient(credentials=credentials)

    with io.open(local_file_path, "rb") as f:
        input_content = f.read()
    features = [videointelligence.Feature.LOGO_RECOGNITION]

    operation = client.annotate_video(
        request={"features": features, "input_content": input_content}
    )

    print("Waiting for operation to complete...")
    response = operation.result()

    # Get the first response, since we sent only one video.
    annotation_result = response.annotation_results[0]

    # Annotations for list of logos detected, tracked and recognized in video.
    for logo_recognition_annotation in annotation_result.logo_recognition_annotations:
        entity = logo_recognition_annotation.entity

        # Opaque entity ID. Some IDs may be available in [Google Knowledge Graph
        # Search API](https://developers.google.com/knowledge-graph/).
        print("Entity Id : {}".format(entity.entity_id))

        print("Description : {}".format(entity.description))

        # All logo tracks where the recognized logo appears. Each track corresponds
        # to one logo instance appearing in consecutive frames.
        for track in logo_recognition_annotation.tracks:
            # Video segment of a track.
            print(
                "\n\tStart Time Offset : {}.{}".format(
                    track.segment.start_time_offset.seconds,
                    track.segment.start_time_offset.microseconds * 1000,
                )
            )
            print(
                "\tEnd Time Offset : {}.{}".format(
                    track.segment.end_time_offset.seconds,
                    track.segment.end_time_offset.microseconds * 1000,
                )
            )
            print("\tConfidence : {}".format(track.confidence))

            # The object with timestamp and attributes per frame in the track.
            for timestamped_object in track.timestamped_objects:

                # Normalized Bounding box in a frame, where the object is located.
                normalized_bounding_box = timestamped_object.normalized_bounding_box
                # print("\n\t\tLeft : {}".format(normalized_bounding_box.left))
                # print("\t\tTop : {}".format(normalized_bounding_box.top))
                # print("\t\tRight : {}".format(normalized_bounding_box.right))
                # print("\t\tBottom : {}".format(normalized_bounding_box.bottom))

                # Optional. The attributes of the object in the bounding box.
                for attribute in timestamped_object.attributes:
                    print("\n\t\t\tName : {}".format(attribute.name))
                    print("\t\t\tConfidence : {}".format(attribute.confidence))
                    print("\t\t\tValue : {}".format(attribute.value))

            # Optional. Attributes in the track level.
            for track_attribute in track.attributes:
                print("\n\t\tName : {}".format(track_attribute.name))
                print("\t\tConfidence : {}".format(track_attribute.confidence))
                print("\t\tValue : {}".format(track_attribute.value))

        # All video segments where the recognized logo appears. There might be
        # multiple instances of the same logo class appearing in one VideoSegment.
        for segment in logo_recognition_annotation.segments:
            print(
                "\n\tStart Time Offset : {}.{}".format(
                    segment.start_time_offset.seconds,
                    segment.start_time_offset.microseconds * 1000,
                )
            )
            print(
                "\tEnd Time Offset : {}.{}".format(
                    segment.end_time_offset.seconds,
                    segment.end_time_offset.microseconds * 1000,
                )
            )


### Text Detection

In [4]:
def video_detect_text(path):

    """Detect text in a local video."""
    credentials = service_account.Credentials.from_service_account_file("/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json")
    video_client = videointelligence.VideoIntelligenceServiceClient(credentials=credentials)
    features = [videointelligence.Feature.TEXT_DETECTION]
    video_context = videointelligence.VideoContext()

    with io.open(path, "rb") as file:
        input_content = file.read()

    operation = video_client.annotate_video(
        request={
            "features": features,
            "input_content": input_content,
            "video_context": video_context,
        }
    )

    print("\nProcessing video for text detection.")
    result = operation.result(timeout=300)

    # The first result is retrieved because a single video was processed.
    annotation_result = result.annotation_results[0]

    for text_annotation in annotation_result.text_annotations:
        print("\nText: {}".format(text_annotation.text))

        # Get the first text segment
        text_segment = text_annotation.segments[0]
        start_time = text_segment.segment.start_time_offset
        end_time = text_segment.segment.end_time_offset
        print(
            "start_time: {}, end_time: {}".format(
                start_time.seconds + start_time.microseconds * 1e-6,
                end_time.seconds + end_time.microseconds * 1e-6,
            )
        )

        print("Confidence: {}".format(text_segment.confidence))

        # Show the result for the first frame in this segment.
        frame = text_segment.frames[0]
        time_offset = frame.time_offset
        print(
            "Time offset for the first frame: {}".format(
                time_offset.seconds + time_offset.microseconds * 1e-6
            )
        )
        print("Rotated Bounding Box Vertices:")
        for vertex in frame.rotated_bounding_box.vertices:
            print("\tVertex.x: {}, Vertex.y: {}".format(vertex.x, vertex.y))

### Save to Pickle file
Takes all features listed in features list and runs video intelligence API to obtain said features. Features can be customized. Face detection done seperately (see bottom) due to runtime.

All output is saved immediately to pickle file, pickle file saved to correspoinding folder by ID

In [5]:
def save_to_pickle(ID):
    from google.oauth2 import service_account
    from google.cloud import vision
    import io
    
    os.chdir(f'{PARENT_DIR}/{ID}')
    path = ID+".mp4"
    pickle_name = f'{ID}_video_intelligence.pickle'
    pickle_dict = {}
    features = [videointelligence.Feature.LABEL_DETECTION, videointelligence.Feature.LOGO_RECOGNITION, videointelligence.Feature.TEXT_DETECTION]

    
    """Detect labels given a file path."""
    credentials = service_account.Credentials.from_service_account_file("/nfs/sloanlab003/projects/arc_proj/story/code/nw/ad-story-arc-b60c485322b4.json")
    video_client = videointelligence.VideoIntelligenceServiceClient(credentials=credentials)
   
    # face detection config - comment out if not using face detection
    config = videointelligence.FaceDetectionConfig(
        include_bounding_boxes=True, include_attributes=True
    )
    context = videointelligence.VideoContext(face_detection_config=config)
    


    with io.open(path, "rb") as movie:
        input_content = movie.read()

    operation = video_client.annotate_video(
        request={"features": features, "input_content": input_content, "video_context": context}
    )
    print("\nProcessing video for logo annotations:", ID)

    result = operation.result(timeout=500)
    print("\nFinished processing.")
    pickle_dict["logo detection"] = list(result.annotation_results[0].logo_recognition_annotations)
    pickle_dict["text detection"] = list(result.annotation_results[0].text_annotations)
    pickle_dict["segment labels"] = list(result.annotation_results[0].segment_label_annotations)
    pickle_dict["shot labels"] = list(result.annotation_results[0].shot_label_annotations)
    pickle_dict["frame labels"] = list(result.annotation_results[0].frame_label_annotations)
    
#     # second round

    with io.open(path, "rb") as f:
        input_content = f.read()

    # Configure the request
    config = videointelligence.FaceDetectionConfig(
        include_bounding_boxes=True, include_attributes=True
    )
    context = videointelligence.VideoContext(face_detection_config=config)

    # Start the asynchronous request
    operation = video_client.annotate_video(
        request={
            "features": [videointelligence.Feature.FACE_DETECTION],
            "input_content": input_content,
            "video_context": context,
        }
    )

    print("\nProcessing video for face detection annotations.")
    result = operation.result(timeout=300)

    print("\nFinished processing.\n")
    
    pickle_dict["facial detection"] = list(result.annotation_results[0].face_detection_annotations)


    with open(f'{PARENT_DIR}/{ID}/{pickle_name}', "wb") as handle:
        pickle.dump(pickle_dict, handle, protocol = pickle.HIGHEST_PROTOCOL)
    print("done saving pickle")

In [ ]:
id_test = 'sb_2023_3515'
os.chdir(f"{PARENT_DIR}/{id_test}")

analyze_labels(id_test+'.mp4')

## Read Pickle

Takes pickle file from previous function (should be saved in corresponding folder by ID) and creates seperate CSV files for each feature.

Logo detection: {ID}_logo_info.csv

Text detection: {ID}_text_info.csv

Face detection: {ID}_faces_info.csv

Label detection (saves 3 seperate CSVs): {ID}_labelvideo_info.csv, {ID}_labelshot_info.csv, {ID}_labelframe_info.csv



In [6]:
def read_pickle_logo(ID):
    os.chdir(f"{PARENT_DIR}/{ID}")
    pickle_name = f'{ID}_video_intelligence.pickle'
    objects = []

    with (open(pickle_name, "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
                
    pickle_dict = objects[0]
    logo_df = {'Entity ID': [], 'Description': [], 'Start Time': [], 'End Time': [], 'Confidence': [], 'Left': [], 'Top': [], 'Right': [], 'Bottom': []}
    
    
    # LOGO detection
    for logo_recognition_annotation in pickle_dict["logo detection"]:
        entity = logo_recognition_annotation.entity
        
        # Opaque entity ID. Some IDs may be available in [Google Knowledge Graph
        # Search API](https://developers.google.com/knowledge-graph/).
        

        # All logo tracks where the recognized logo appears. Each track corresponds
        # to one logo instance appearing in consecutive frames.
        for track in logo_recognition_annotation.tracks:
            logo_df["Entity ID"].append(entity.entity_id)
            logo_df["Description"].append(entity.description)
            logo_df["Start Time"].append(track.segment.start_time_offset.seconds + track.segment.start_time_offset.microseconds * 1000)
            logo_df["End Time"].append(track.segment.end_time_offset.seconds + track.segment.end_time_offset.microseconds * 1000)
            logo_df["Confidence"].append(track.confidence)
            
            timestamped_object = track.timestamped_objects[0]
            normalized_bounding_box = timestamped_object.normalized_bounding_box
            
            logo_df["Left"].append(normalized_bounding_box.left)
            logo_df["Top"].append(normalized_bounding_box.top)
            logo_df["Right"].append(normalized_bounding_box.right)
            logo_df["Bottom"].append(normalized_bounding_box.bottom)

    newdf = pd.DataFrame(data=logo_df) 
    os.chdir(f"{PARENT_DIR}/{ID}")
    newdf.to_csv(f"{ID}_logo_info.csv")
    print('done with saving logo info')
    
def read_pickle_text(ID):
    os.chdir(f"{PARENT_DIR}/{ID}")
    pickle_name = f'{ID}_video_intelligence.pickle'
    objects = []

    with (open(pickle_name, "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    pickle_dict = objects[0]
    
    text_df = {'Text': [], 'Start Time': [], 'End Time': [], 'Confidence': [], 'Time Offset': [], 'Vertex 1': [], 'Vertex 2': [], 'Vertex 3': [], 'Vertex 4': []}
    
    for text_annotation in pickle_dict["text detection"]:
        
        text_df['Text'].append(text_annotation.text)

        # Get the first text segment
        text_segment = text_annotation.segments[0]
        start_time = text_segment.segment.start_time_offset
        end_time = text_segment.segment.end_time_offset
        
        text_df['Start Time'].append(start_time.seconds + start_time.microseconds * 1e-6)
        text_df['End Time'].append(end_time.seconds + end_time.microseconds * 1e-6)
        text_df['Confidence'].append(text_segment.confidence)

        # Show the result for the first frame in this segment.
        frame = text_segment.frames[0]
        time_offset = frame.time_offset
        text_df['Time Offset'].append(time_offset.seconds + time_offset.microseconds * 1e-6)

        for i, vertex in enumerate(frame.rotated_bounding_box.vertices):
            index = i+1
            text_df[f'Vertex {index}'].append((vertex.x, vertex.y))
        
    newdf = pd.DataFrame(data=text_df)
    os.chdir(f"{PARENT_DIR}/{ID}")
    newdf.to_csv(f"{ID}_text_info.csv")
    print('done with saving text info')
    
def read_pickle_face(ID):
    os.chdir(f"{PARENT_DIR}/{ID}")
    pickle_name = f'{ID}_video_intelligence.pickle'
    objects = []

    with (open(pickle_name, "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    pickle_dict = objects[0]
    face_df = {'Start Time': [], 'End Time': [], 'Left': [], 'Top': [], 'Right': [], 'Bottom': [], 'glasses': [], 'headwear': [], 'eyes_visible': [], 'mouth_open': [], 'looking_at_camera': [], 'smiling': []}
    
    for annotation in pickle_dict["facial detection"]:
        # print(annotation)
        for track in annotation.tracks:
            face_df["Start Time"].append(track.segment.start_time_offset.seconds + track.segment.start_time_offset.microseconds / 1e6)
            face_df["End Time"].append(track.segment.end_time_offset.seconds + track.segment.end_time_offset.microseconds / 1e6)
            
            
            timestamped_object = track.timestamped_objects[0]
            box = timestamped_object.normalized_bounding_box
            
            face_df["Left"].append(box.left)
            face_df["Top"].append(box.top)
            face_df["Right"].append(box.right)
            face_df["Bottom"].append(box.bottom)
            
            # print(timestamped_object.attributes)
            for attribute in timestamped_object.attributes:
                # print(attribute)
                face_df[str(attribute.name)].append(attribute.confidence)
    
    if len(face_df["Start Time"]) != len(face_df["glasses"]):
        difference = len(face_df["Start Time"]) - len(face_df["glasses"])
        # print(face_df.items()[0])
        for column, listt in list(face_df.items())[6:]:
            listt.extend([None]*difference)
            print(column, len(listt))
        
    newdf = pd.DataFrame(data=face_df)
    os.chdir(f"{PARENT_DIR}/{ID}")
    newdf.to_csv(f"{ID}_faces_info.csv", index=True)
    print('done with saving face info', ID)
    
def read_pickle_label(ID):
    os.chdir(f"{PARENT_DIR}/{ID}")
    pickle_name = f'{ID}_video_intelligence.pickle'
    objects = []

    with (open(pickle_name, "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    pickle_dict = objects[0]
    
    video_df = {'Description': [], 'Label Category Description': [], 'Time Start': [], 'Time End': [], 'Confidence': []}
    shot_df = {'Description': [], 'Label Category Description': [], 'Time Start': [], 'Time End': [], 'Confidence': []}
    frame_df = {'Description': [], 'Label Category Description': [], 'Frame Time Offset': [], 'Confidence': []}
    
    for segment_label in pickle_dict["segment labels"]:
        category = ''
        if len(segment_label.category_entities) != 0:
            category = segment_label.category_entities[0].description
        for segment in segment_label.segments:
            video_df['Description'].append(segment_label.entity.description)
            video_df['Label Category Description'].append(category)
            video_df['Time Start'].append(segment.segment.start_time_offset.seconds + segment.segment.start_time_offset.microseconds / 1e6)
            video_df['Time End'].append(segment.segment.end_time_offset.seconds + segment.segment.end_time_offset.microseconds / 1e6)
            video_df['Confidence'].append(segment.confidence)
                
    for shot_label in pickle_dict["shot labels"]:
        category = ''
        if len(shot_label.category_entities) != 0:
            category = shot_label.category_entities[0].description
        for shot in shot_label.segments:
            shot_df['Description'].append(shot_label.entity.description)
            shot_df['Label Category Description'].append(category)
            shot_df['Time Start'].append(shot.segment.start_time_offset.seconds + shot.segment.start_time_offset.microseconds / 1e6)
            shot_df['Time End'].append(shot.segment.end_time_offset.seconds + shot.segment.end_time_offset.microseconds / 1e6)
            shot_df['Confidence'].append(shot.confidence)
        
    for frame_label in pickle_dict["frame labels"]:
        category = ''
        if len(frame_label.category_entities) != 0:
            category = frame_label.category_entities[0].description
        for frame in frame_label.frames:
            shot_df['Description'].append(frame_label.entity.description)
            shot_df['Label Category Description'].append(category)
            shot_df['Frame Time Offset'].append(frametime_offset.seconds + frame.time_offset.microseconds / 1e6)
            shot_df['Confidence'].append(frame.confidence)
                
    videodf = pd.DataFrame(data=video_df)
    shotdf = pd.DataFrame(data=shot_df)
    framedf = pd.DataFrame(data=frame_df)
    os.chdir(f"{PARENT_DIR}/{ID}")
    videodf.to_csv(f"{ID}_labelvideo_info.csv", index=True)
    shotdf.to_csv(f"{ID}_labelshot_info.csv", index=True)
    framedf.to_csv(f"{ID}_labelframe_info.csv", index=True)
    
    print('done with saving label info', ID)



In [ ]:
ID = 'sb_2023_3515'
os.chdir(f"{PARENT_DIR}/{ID}")
# detect_person(ID+'.mp4')
read_pickle_label(ID)
# read_pickle_text(ID)
# read_pickle_logo(ID)

# Run Everything

In [7]:
CSV_NAME = "nw_may_v1.csv"
os.chdir(f"{LINK_DIR}")
df = pd.read_csv(CSV_NAME)

def run_all(df):
    for i in range(1500, df.shape[0]):
        
        if df["has_vid"][i] == 0:
            continue
        
        ID = df['id'][i]
        
        save_to_pickle(ID)
        read_pickle_face(ID)
        read_pickle_text(ID)
        read_pickle_logo(ID)
        read_pickle_label(ID)

def run_one(ID):
    save_to_pickle(ID)
    read_pickle_face(ID)
    read_pickle_text(ID)
    read_pickle_logo(ID)
    read_pickle_label(ID)
    
run_one('sb_2002_1426')


Processing video for logo annotations: sb_2002_1426

Finished processing.

Processing video for face detection annotations.

Finished processing.

done saving pickle
done with saving face info sb_2002_1426
done with saving text info
done with saving logo info
done with saving label info sb_2002_1426
